In [1]:
import serial
from serial.tools.list_ports import comports
import time

from imagewriter.debug import FlowControlLogger
from imagewriter.encoding.attributes import START_DOUBLE_WIDTH, STOP_DOUBLE_WIDTH
from imagewriter.encoding.language import Language
from imagewriter.encoding.motion import FF
from imagewriter.encoding.switch import SoftwareSwitch
import imagewriter.serial
from tests.fixtures import HELLO_WORLD

PORT = '/dev/cu.PL2303G-USBtoUART1120' # Depends on system
BAUD_RATE = 9600 # Depends on DIP switches

conn = None
flow_control_logger = None

In [2]:
def list_ports():
    return [ port.device for port in comports()]


def connect(subclass=False, log=True):
    global conn
    global flow_control_logger

    if conn:
        conn.close()

    if subclass:
        conn = imagewriter.serial.Serial(
            port=PORT,
            baudrate=BAUD_RATE
        )
    else:
        conn = serial.Serial(
            port=PORT,
            baudrate=BAUD_RATE,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            dsrdtr=True,
            rtscts=True,
            xonxoff=False
        )

    running = flow_control_logger.running if flow_control_logger else False

    if flow_control_logger:
        flow_control_logger.serial = conn
        
    else:
        flow_control_logger = FlowControlLogger(conn)

    if log and not running:
        flow_control_logger.start()


def close():
    global conn
    global flow_control_logger
    
    conn.close()
    flow_control_logger.shutdown()
    conn = None
    flow_control_logger = None


def toggle():
    conn.rts = False
    time.sleep(0.1)
    conn.rts = True


def disable_rtscts():
    conn.rtscts = False


def enable_rtscts():
    conn.rtscts = True


def config():
    commands = SoftwareSwitch.toggle(
        SoftwareSwitch.LANGUAGE_1,
        SoftwareSwitch.LANGUAGE_2,
        SoftwareSwitch.LANGUAGE_3,
        # SoftwareSwitch.SOFTWARE_SELECT_RESPONSE_DISABLED,
        # SoftwareSwitch.LF_WHEN_LINE_FULL,
        # SoftwareSwitch.PRINT_COMMANDS_INCLUDE_LF_FF,
        # SoftwareSwitch.AUTO_LF_AFTER_CR,
        # SoftwareSwitch.SLASHED_ZERO,
        # SoftwareSwitch.PERFORATION_SKIP_DISABLED,
        # SoftwareSwitch.IGNORE_EIGHTH_DATA_BIT,
        # *SoftwareSwitch.language_switches(Language.AMERICAN)
    )
    for cmd in commands:
        conn.write(bytes(cmd))
    conn.write(b"\r")

In [ ]:
connect(subclass=False, log=True)

In [ ]:
flow_control_logger.start()

In [ ]:
flow_control_logger.stop()

In [3]:
config()

close command:
0b11100000 0b0


In [ ]:
i = 0

while conn.cts and i < 2048:
    conn.write(b"x")
    i += 1

print(f"Printer accepted {i} bytes before disabling CTS")

disable_rtscts()

conn.write(b"Hello!\r\n")

enable_rtscts()

toggle()

In [ ]:
conn.write(b"Hello world!\r\n")

In [ ]:
toggle()

In [ ]:
conn.rts = False

In [ ]:
conn.rts = True

In [ ]:
close()

In [ ]:
conn.close()

In [ ]:
conn.open()

In [ ]:
print("8765432187654321")
print(bin(SoftwareSwitch.LANGUAGE_1.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.LANGUAGE_2.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.LANGUAGE_3.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.SOFTWARE_SELECT_RESPONSE_DISABLED.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.LF_WHEN_LINE_FULL.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.PRINT_COMMANDS_INCLUDE_LF_FF.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.AUTO_LF_AFTER_CR.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.SLASHED_ZERO.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.PERFORATION_SKIP_DISABLED.value)[2:].rjust(16, "|"))
print(bin(SoftwareSwitch.IGNORE_EIGHTH_DATA_BIT.value)[2:].rjust(16, "|"))

In [ ]:
bin(2)

In [9]:
import struct

def print_switch_bank(bank: int):
    print("12345678")
    print(bin(bank)[2:].replace("0", "|").replace("1", "x").rjust(8, "|"))

def in_binary(n: int):
    return bin(n)[2:].rjust(16, "0")

switches = {
    SoftwareSwitch.LANGUAGE_1,
    # SoftwareSwitch.LANGUAGE_2,
    # SoftwareSwitch.LANGUAGE_3,
    SoftwareSwitch.SOFTWARE_SELECT_RESPONSE_DISABLED,
    SoftwareSwitch.LF_WHEN_LINE_FULL,
    SoftwareSwitch.PRINT_COMMANDS_INCLUDE_LF_FF,
    SoftwareSwitch.AUTO_LF_AFTER_CR,
    # SoftwareSwitch.SLASHED_ZERO,
    SoftwareSwitch.PERFORATION_SKIP_DISABLED,
    SoftwareSwitch.IGNORE_EIGHTH_DATA_BIT,
}

switches = {
    # SoftwareSwitch.LANGUAGE_1,
    SoftwareSwitch.LANGUAGE_2,
    SoftwareSwitch.LANGUAGE_3,
    # SoftwareSwitch.SOFTWARE_SELECT_RESPONSE_DISABLED,
    # SoftwareSwitch.LF_WHEN_LINE_FULL,
    # SoftwareSwitch.PRINT_COMMANDS_INCLUDE_LF_FF,
    # SoftwareSwitch.AUTO_LF_AFTER_CR,
    SoftwareSwitch.SLASHED_ZERO,
    # SoftwareSwitch.PERFORATION_SKIP_DISABLED,
    # SoftwareSwitch.IGNORE_EIGHTH_DATA_BIT,
}

bank_a = 0
bank_b = 0

def algorithm_a():
    global bank_a
    global bank_b
    
    for sw in sorted(switches, key=lambda s: s.value):
        v = sw.value
        b = 0
        # for i in range(1, 9):
        #     print(256 >> i)
        # print('---')
        # for i in range(0, 8):
        #     print(1 << i)
        # print('---')
        for i in range(0, 16):
            # ith bit
            bit = v & (1 << i)
    
            if bit:
                swapped = 0x8000 >> i
                if i < 8:
                    swapped = swapped >> 8
                    bank_a |= swapped
                else:
                    bank_b |= swapped

def algorithm_b():
    global bank_a
    global bank_b

    big_endian = 0
    
    for sw in sorted(switches, key=lambda s: s.value):
        big_endian |= sw.value

    for i in range(0, 16):
        bit = big_endian & (1 << i)
    
        if bit:
            swapped = 0x8000 >> i
            if i < 8:
                swapped = swapped >> 8
                bank_a |= swapped
            else:
                bank_b |= swapped

algorithm_b()

print("  12345678 12345678")
print(f"0b{bin(bank_a)[2:]} {bin(bank_b)[2:]}")

buffer = bytes([bank_a, bank_b])
print(bin(buffer[0]))
print(bin(buffer[1]))

  12345678 12345678
0b1100000 10000000
0b1100000
0b10000000


In [ ]:
print(bin(0x80))

In [ ]:
hex(0b10000000)

In [ ]:
hex(0x8000 >> 8)

In [10]:
b"ab"[0:1]

b'a'